# Case study: Reed breast leukocyte data

This notebook generates results for a case study that uses a [Human Breast Cell Atlas dataset](https://cellxgene.cziscience.com/collections/48259aa8-f168-4bf5-b797-af8e88da6637).

This is an example where we use the Extended Neighbourhood-Proportion-Error (xNPE) to compare embedding distortion of different cell populations, as well as neighbourhood composition plots to reveal sources of distortion.

<hr>

## **0.** Load required modules

In addition to `ViVAE`, `ViScore` and their dependenceis, you will need to have `umap` installed in your conda environment.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import copy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import vivae as vv
import viscore as vs
from sklearn.manifold import TSNE
from umap import UMAP
import torch
import anndata as ad
import scanpy as sc
import flowsom as fs

## **1.** Load input data

See the ViScore [benchmarking](https://github.com/saeyslab/ViScore/tree/main/benchmarking) page for instructions on how to obtain and pre-process the Reed data.

In [ ]:
## Load input data

dataset = 'Reed'

hd    = sc.read_h5ad(f'./{dataset}.h5ad')
pc    = np.load(os.path.join('..', 'data', f'{dataset}_input.npy'),          allow_pickle=True)
pc_d  = np.load(os.path.join('..', 'data', f'{dataset}_input_denoised.npy'), allow_pickle=True)
labs  = np.load(os.path.join('..', 'data', f'{dataset}_labels.npy'),         allow_pickle=True)
unass = np.load(os.path.join('..', 'data', f'{dataset}_unassigned.npy'),     allow_pickle=True).item()

We will also generate (or load) a *k*-nearest-neighbour graph for 1000 neighbours, to use for the xNPE computation.

In [ ]:
knn1000 = vs.make_knn(pc, fname=os.path.join('..', 'data', f'{dataset}_knn1000.npy'), k=1000, random_state=42)

## **2.** Generating embeddings

We use *t*-SNE, UMAP and ViVAE to generate embeddings of this dataset.
We use default hyperparameter values, with denoising for *ViVAE*.
(See our paper for ablation studies about denoising.)

We use the `random_state` argument in model constructors, and manual seeds for `numpy` and `torch`.
This should achieve reproducibility across runs on a single machine (with the same GPU/CPU device for *ViVAE*).
If complete reproducibility (*ie.*, identical results) is not achieved with *ViVAE*, consult [this document](https://pytorch.org/docs/stable/notes/randomness.html) for troubleshooting.

In [ ]:
## Train t-SNE model & transform inputs

model_tsne = TSNE(n_components=2, random_state=1)
emb_tsne = model_tsne.fit_transform(pc)
np.save('Reed_emb_tsne.npy', emb_tsne, allow_pickle=True)

In [ ]:
## Train UMAP model & transform inputs

model_umap = UMAP(n_components=2, random_state=1)
emb_umap = model_umap.fit_transform(pc)
np.save('Reed_emb_umap.npy', emb_umap, allow_pickle=True)

In [ ]:
## Train ViVAE model & transform inputs

np.random.seed(1)
torch.manual_seed(1)
model_vivae = vv.ViVAE(input_dim=pc.shape[1], latent_dim=2, random_state=1)
model_vivae.fit(pc_d)
emb_vivae = model_vivae.transform(pc_d)
np.save('Reed_emb_vivae_seed1.npy', emb_vivae, allow_pickle=True)

In [ ]:
## Generate 4 more ViVAE embeddings to test changes due to stochasticity

for irun in [2,3,4,5]:
    np.random.seed(irun)
    torch.manual_seed(irun)
    model_vivae = vv.ViVAE(input_dim=pc.shape[1], latent_dim=2, random_state=irun)
    model_vivae.fit(pc_d, verbose=False)
    emb_vivae = model_vivae.transform(pc_d)
    np.save(f'Reed_emb_vivae_seed{irun}.npy', emb_vivae, allow_pickle=True)

In [ ]:
emb_vivae = np.load('Reed_emb_vivae_seed1.npy', allow_pickle=True)

## **3.** Compute population-level xNPE scores

We use xNPE scoring to quantify misembedding of labelled cell populations using the 1000-nearest-neighbour graph built earlier.

In [ ]:
## Compute xNPE

xnpe_tsne  = vs.xnpe(hd=pc, ld=emb_tsne,  annot=labs, knn=knn1000, random_state=42)
xnpe_umap  = vs.xnpe(hd=pc, ld=emb_umap,  annot=labs, knn=knn1000, random_state=42)
xnpe_vivae = vs.xnpe(hd=pc, ld=emb_vivae, annot=labs, knn=knn1000, random_state=42)

In [ ]:
## Save xNPE results

np.save('Reed_xnpe_tsne.npy',  xnpe_tsne, allow_pickle=True)
np.save('Reed_xnpe_umap.npy',  xnpe_umap, allow_pickle=True)
np.save('Reed_xnpe_vivae.npy', xnpe_vivae, allow_pickle=True)

## **4.** Plot generated embeddings

Using a palette or more-or-less distinctive colours, we plot the 2-d layouts of the data that we obtained from the three methods.

In [ ]:
palette = {
    'CD4-positive helper T cell':                           'sandybrown',
    'CD8-positive, alpha-beta memory T cell':               '#c1c86c',
    'Tc1 cell':                                             '#699dc0',
    'class switched memory B cell':                         '#d889c1',
    'dendritic cell':                                       '#a89ef4',
    'effector memory CD8-positive, alpha-beta T cell':      'firebrick',
    'innate lymphoid cell':                                 '#757469',
    'macrophage':                                           'olive',
    'mature NK T cell':                                     '#87d7d6',
    'naive B cell':                                         '#6afd6e',
    'naive thymus-derived CD4-positive, alpha-beta T cell': 'rebeccapurple',
    'natural killer cell':                                  'chocolate',
    'plasma cell':                                          'black',
    'unknown':                                              '#74c7c0',
    'unswitched memory B cell':                             'goldenrod' 
}

In [ ]:
mpl.rcParams['axes.linewidth'] = 0.1
pops = list(palette.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(5, 1.75), dpi=150)
fig.subplots_adjust(wspace=.05)
handles = []
labels = []
embs = [emb_tsne, emb_umap, emb_vivae]
s = .01
for i, method in enumerate(methods):
        idcs0 = np.where(labs==unass)[0]
        idcs1 = np.delete(np.arange(len(labs)), idcs0)
        emb = embs[i]
        ax[i].scatter(emb[idcs0,0], emb[idcs0,1], s=s, c='#bfbfbf', alpha=1., linewidths=0)
        idx_pop = 0
        l = np.delete(labs, idcs0)
        for pop in np.unique(l):
            idcs = np.where(l == pop)[0]
            if i==(len(methods)-1):
                ax[i].scatter(emb[idcs1[idcs],0], emb[idcs1[idcs],1], label=pop, s=s, c=palette[pop], alpha=1., linewidth=.5)
            else:
                ax[i].scatter(emb[idcs1[idcs],0], emb[idcs1[idcs],1], s=s, c=palette[pop], alpha=1., linewidth=.5)
            ax[i].tick_params(axis='both', labelbottom=False, labelleft=False, bottom=False, left=False)
            idx_pop += 1
        ax[i].axis('equal')
        ax[i].set_title(f'  {methods[i]}', size=7, ha='left', x=-.03, y=.98)
        if i==(len(methods)-1):
            # Re-order points in legend to make them more sensible (ad hoc!)
            handles, labels = plt.gca().get_legend_handles_labels()
            order = [0,1,2,5,10,8,11,4,9,13,3,12,7,6]
            
            fig.legend([handles[idx] for idx in order],[labels[idx] for idx in order], bbox_to_anchor=(1.35, .92), fontsize=5, markerscale=50.)

fig.savefig('Reed_A_embeddings.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('Reed_A_embeddings.svg', bbox_inches='tight', transparent=True)

## **5.** Plot xNPE and neighoburhood compositions

Next, we generate diagnostic plots.
We pick a population of interest (`poi`: `'natural killer cell`), highlight it in the embeddings, show xNPE values for it and all other populations (for the three different methods) and create neighbourhood composition plots that show sources of embedding error for this POI.

In [ ]:
## Set population of interest (POI)

poi = 'natural killer cell'

In [ ]:
## Highlight the POI in 2-d layouts

mpl.rcParams['axes.linewidth'] = 0.1
pops = list(xnpe_tsne.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(5, 1.75), dpi=150)
fig.subplots_adjust(wspace=.05)
handles = []
labels = []
embs = [emb_tsne, emb_umap, emb_vivae]
s = .01

idcs_nk = np.where(labs==poi)[0]
for i, method in enumerate(methods):

        emb = embs[i]
        ax[i].scatter(emb[:,0], emb[:,1], s=s, c='grey', alpha=1., linewidth=.5)
        ax[i].scatter(emb[idcs_nk,0], emb[idcs_nk,1], s=1., c=palette[poi], alpha=1., linewidth=.5)

        ax[i].axis('equal')
        ax[i].tick_params(axis='both', labelbottom=False, labelleft=False, bottom=False, left=False)
fig.savefig('reed_B_embeddings_poi.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_B_embeddings_poi.svg', bbox_inches='tight', transparent=True)

In [ ]:
## Aggregate xNPE values for all populations for all 3 embeddings

pops = list(xnpe_tsne.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
d = pd.DataFrame(
    data=np.vstack([list(xnpe_tsne.values()), list(xnpe_umap.values()), list(xnpe_vivae.values())]),
    columns=pops
)
d.index = methods
d = d.drop(unass, axis=1)
pops = list(d.keys())
npop = len(pops)

In [ ]:
d

In [ ]:
## Plot the xNPE values using a barplot

fig, ax = plt.subplots(figsize=(2,4), dpi=150)
mpl.rcParams['axes.linewidth'] = 0.1
pal = ['orange', 'teal', 'darkred']
n = npop*(nmet+1)

for i, method in enumerate(methods):
    for j, pop in enumerate(pops):
        if j==0:
            ax.barh(y=j*(nmet+1)+i, width=d[pop][method], color=pal[i], label=method)
        else:
            ax.barh(y=j*(nmet+1)+i, width=d[pop][method], color=pal[i])
ax.set_xlim((0., 1.2))
ax.set_ylim((-1, n))
pop_labels = copy.deepcopy(pops)
ax.set_yticks(ticks=np.arange(npop)*(nmet+1)+1, labels=pop_labels)
ax.tick_params(axis='both', labelsize=6)
plt.gca().invert_yaxis()

fig.legend(bbox_to_anchor=(.88, .845), fontsize=6)
fig.savefig('reed_C_barplot.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_C_barplot.svg', bbox_inches='tight', transparent=True)

In [ ]:
## Calculate neighbourhood compositions per population per embedding

pop      = poi
nc_hd    = vs.neighbourhood_composition(X=pc, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_tsne  = vs.neighbourhood_composition(X=emb_tsne, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_umap  = vs.neighbourhood_composition(X=emb_umap, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_vivae = vs.neighbourhood_composition(X=emb_vivae, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)

In [ ]:
nc_vivae_all = [vs.neighbourhood_composition(X=np.load(fname, allow_pickle=True), pop=pop, annot=labs, k=1000, stepsize=10, random_state=42) for fname in [f'Reed_emb_vivae_seed{seed}.npy' for seed in [1,2,3,4,5]]]

In [ ]:
## Plot neighbourhood compositions

palette_without_poi = np.array(list(palette.values()))[np.where([x!=pop for x in list(palette.keys())])[0]]

fig, ax = plt.subplots(ncols=1, nrows=4, figsize=(3, 5), dpi=150, sharey=True, sharex=True)
mpl.rcParams['axes.linewidth'] = 0.1
fig.subplots_adjust(hspace=.35)

for i, nc in enumerate([nc_hd, nc_tsne, nc_umap, nc_vivae]):
    y = np.vstack(nc[3])
    ax[i].stackplot(nc[4], y.T, labels=nc[1], colors=palette_without_poi)
    ax[i].set_ylim((0.,1.))
    ax[i].set_xlim((min(nc[4]), max(nc[4])))
    ax[i].set_title(['HD data', 't-SNE', 'UMAP', 'ViVAE'][i], size=7, ha='left', x=.0, y=.96)
    ax[i].tick_params(axis='both', labelsize=5)

ax[3].set_ylabel('Neighbour proportion', loc='bottom', size=7)
ax[3].set_xlabel('Neighbourhood rank', loc='left', size=7)

fig.savefig('reed_D_ncp.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_D_npc.svg', bbox_inches='tight', transparent=True)